# Distributed Parallel Programming using MPI for Python

MPI for Python supports convenient, pickle-based communication of generic Python object as well as fast, near C-speed, direct array data communication of buffer-provider objects (e.g., NumPy arrays). The former is implemented on top of the latter.

In MPI, there are two types of data supported: The Native python types (such as pickle, send, etc.), and the buffers (such as Numpy arrays). The native python types are slower, but more convenient. The buffers are faster, but less convenient. The native python types are implemented on top of the buffers.

Maybe the most important change in MPI for python is the use of communicator objects as member functions instead of module-level functions. This allows multiple MPI worlds to coexist in the same program, and allows the use of MPI in a thread-safe manner.

The following example shows how to use MPI for Python to computing Pi in Python, in a sequential way:

In [ ]:
from math import pi
from time import time

tstart = time()
n = 10000000
s = 0.0
width = 1.0 / n

for i in range(0, n):
    x = (i + 0.5) * width
    f_x = s + 4.0 / (1.0 + x * x)
    s += f_x
tend = time()

print("pi is approximately %.16f, Error is %.16f" % (s * width, abs(s * width - pi)))
print("Time: %.16f seconds" % (tend - tstart))

And, using MPI, this code looks like:

In [ ]:
from mpi4py import MPI  # MPI_INIT is called during import
from math import pi
from time import time

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

tstart = time()
n = 10000000
width = 1.0 / n
sum = 0.0

for i in range(rank, n, size):
    x = (i + 0.5) * width
    sum += 4.0 / (1.0 + x * x)

tend = time()

pi_teo = comm.reduce(sum, op=MPI.SUM)  # Reduce with SUM operation, it will be stored results in rank 0, by summing all the results from all the ranks, by default it sends to rank 0 and tag 0

if rank == 0:
    print("pi is approximately %.16f, Error is %.16f" % (pi_teo * width, abs(pi_teo * width - pi)))
    print("Time: %.16f seconds" % (tend - tstart))

It is important to note that MPI_Finalize is called automatically when the module is unloaded. This means that MPI programs written in Python do not need to call MPI_Finalize explicitly. However, MPI programs written in Python should not rely on the automatic finalization of MPI. Instead, MPI_Finalize should be called explicitly when it is desired to exit the MPI program. This is especially important when using MPI within a GUI application, such as Tkinter or wxPython, since the GUI event loop may be running after the main program has exited.

**Moreover, if you want that the value from reduce is stored in all the ranks, you can use the allreduce function. It will store the value in all the ranks, not only in rank 0.**

**It is recommended to do not install mpi4py in a venv, instead of on your root home. This is because mpi4py is a experimental package, and it is updated frequently.**

# Ring problem

In this case, we will have 5 processes in a ring way, where there are a message on the process 0, with values of 1 or 0. Our task is to create a program that pass the value to the all processes. Take care about using send or receive commands, because it can cost you a lot of time. However, using broadcasting also will cost a lot of time (scales with log).

It is important to state what are blocking and non-blocking messages.

HOMEWORK: Do the same ring problem, but using two messages. One of the messages goes in forward direction, and the second goes in backward direction. IA RECOMENDATION: You can use the function sendrecv for this purpose.

Se requiere que el mensaje llegue a todos los procesos, y que estos procesos sean los que se impriman.

# Shared Memory Programming Paradigm

The shared memory programming paradigm is a model for parallel programming in which a large virtual memory space is shared among the processes of a parallel program. The processes communicate with each other by reading and writing data to the shared memory space. Every thread on the cluster have a part of the information located in a shared memory, available for all threads, and a "private" part of the information, that is only available for the thread. 

In this context, a master thread is a thread responsible to coordinate the other threads available in the machine. Therefore, using tools as OpenMp will be useful to create a master thread, and the other threads will be the slaves.

OpenMP is an Application Program Interface (API), jointly defined by a group of major computer hardware and software vendors. OpenMP provides a portable, scalable model for developers of shared memory parallel applications. The API supports C/C++ and Fortran on a wide variety of architectures. OpenMP offers the following benefits to developers:

- It is directive based, so it is easy to learn and use.
- OpenMP only works in shared memory systems.
- OpenMP hides the calls to a thread library, so it is portable.
- OpenMP can be added incrementally to existing applications.

In MPI, for example, to convert a code into parallel we will need to re-write the code, and to use the functions of MPI (something painful if you for example have 20k lines of code). However, in OpenMP, we only need to add some directives to the code, and it will be parallelized. However, OpenMP can be using in a single machine, and MPI can be used in a cluster of machines.

A simple C program to make a Hello world in OpenMP follows the example of Ivan slides.

In OpenMP, it is possible to schedule the work in a static or dynamic way. In the static way, the work is divided in equal parts, and each thread will work in a part of the problem. In the dynamic way, the work is divided in equal parts, but each thread will work in a part of the problem, and when it finishes, it will take another part of the problem. The dynamic way is useful when the work is not equally distributed.

In OpenMP, it is possible to use the Reduction operation, that is a way to sum the results of each thread, and to store the result in a variable. It is important because the variable in reduction is private for each thread, and the result is stored in a shared variable. Therefore, they are synchronized in the operation, avoiding overwriting the results.

In OpenMP, the code starts in a sequential way, and you can decide where to parallelize the code. For example, if you have a for loop, you can parallelize the for loop, and the code will be parallelized. However, if you have an if statement before the OpenMP implementation, the code will be executed in a sequential way. Therefore, it is important to take care about the parallelization of the code.



# A Parallel code for the LJ Model for Argon

In this case, we will use the LJ model for Argon, and we will parallelize the code using OpenMP and MPI. The code is based on the Lennard-Jones model, that is a potential that describes the interaction between a pair of neutral atoms or molecules. The Lennard-Jones potential is given by:

$$ V = \sum_{i=1}^{N-1} \sum_{j=i+1}^{N} 4 \epsilon \left[ \left( \frac{\sigma}{r_{ij}} \right)^{12} - \left( \frac{\sigma}{r_{ij}} \right)^{6} \right] $$

where $r_{ij}$ is the distance between the atoms $i$ and $j$, $\epsilon$ is the depth of the potential well, and $\sigma$ is the finite distance at which the inter-particle potential is zero.

The order of the algorithm is:

1. Read in parameters of initial state of particles
2. Integrate motion equations with Verlet velocity for a given number of steps. The idea is propagated all velocities for half a time step, then propagate all positions for a full time step, and then propagate all velocities for half a time step (by calculating forces). The advantage of using this approach consists in the cancellation of the second order error in the velocity.
3. Check if the total energy is conserved in all the routine.

To run, you can use the following command (locate the Makefile inside the folder):

```bash
make
```

Then, see for the examples folder (it will create 3 .x files), and use:
```
../ljmd-serial.x < argon_2916.inp
```

And it will run the ljmd-serial code using as input argon_2916.inp.
